# Group Details

## Group Name:

### Student 1:

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [10]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [11]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [12]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""

example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [13]:

example = load_array('data/task 1/train/trajectory_1.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

shape of initial state (model input): (5, 5)
shape of terminal state (to be predicted by model): (5, 2)


In [14]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time. 
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-1.11611543  3.21149953         nan         nan]
 [-0.2865083   4.30801877         nan         nan]
 [ 1.07701594 -8.12529269         nan         nan]
 [-0.92053478  3.13709551         nan         nan]
 [-3.96308297 -4.27733589         nan         nan]
 [ 2.33945401 -8.67733599         nan         nan]
 [-4.83949085  3.67854952         nan         nan]
 [ 0.31080159 -9.74720071         nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import os

In [16]:
#Read data from multiple files
initial_state_train=[]
terminal_state_train=[]
for i in range(900):
    data =load_array('data/task 1/train/trajectory_'+str(i)+'.npz', task='task 1')
    initial_state_train.append(data[0])
    terminal_state_train.append(data[1])



In [17]:
#find the max length of the data
max_length=0
for i in range(900):
    if initial_state_train[i].shape[0]>max_length:
        max_length=initial_state_train[i].shape[0]

# Data Handling and Preprocessing

In [18]:
#build a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.file_list = os.listdir(data_dir)

    def __getitem__(self, index):
        file_path = os.path.join(self.data_dir, self.file_list[index])
        # Read and preprocess the data from the file
        data = self._read_file(file_path)
        # Return the preprocessed data
        return data

    def __len__(self):
        return len(self.file_list)

    def _read_file(self, file_path):
        data= load_array(file_path, task='task 1')
        # Pad the data with zeros so that all sequences have length 9
        input_tensor= np.pad(data[0], ((0,9-len(data[0])),(0,0)), 'constant', constant_values=(0))
        target_tensor= np.pad(data[1], ((0,9-len(data[0])),(0,0)), 'constant', constant_values=(0))


        input_tensor = torch.tensor(input_tensor)
        target_tensor = torch.tensor(target_tensor)
        return input_tensor, target_tensor 


In [19]:
data_dir = 'data/task 1/train'

In [20]:
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Model Implementation

In [37]:
class DeepSet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepSet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.aggregator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        aggregated = torch.sum(encoded, dim=0)
        aggregated = self.aggregator(aggregated)
        output = self.decoder(aggregated)
        return output



In [38]:
input_dim = 5
hidden_dim = 64
output_dim = 2  # Output x and y coordinates
batch_size = 32
num_epochs = 10

# Model Training

In [39]:
model = DeepSet(input_dim, hidden_dim, output_dim)

# Create the data loader
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [40]:
for epoch in range(num_epochs):
    for input_tensor, target_tensor in data_loader:
        #set input and target to float
        input = input_tensor.to('cpu').float()
        target = target_tensor.to('cpu').float()

        # Forward pass
        output = model(input).to('cpu')
        # Compute the loss
        loss = criterion(output, target)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

d:\DSAI-22-24\TUe-Year1\Q4\Deep Learning\tue-deeplearning\.conda\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32, 9, 2])) that is different to the input size (torch.Size([9, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
d:\DSAI-22-24\TUe-Year1\Q4\Deep Learning\tue-deeplearning\.conda\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([4, 9, 2])) that is different to the input size (torch.Size([9, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Loss: 11.836203575134277
Epoch [2/10], Loss: 10.414335250854492
Epoch [3/10], Loss: 18.0449161529541
Epoch [4/10], Loss: 18.16114616394043
Epoch [5/10], Loss: 12.758383750915527
Epoch [6/10], Loss: 11.87692642211914
Epoch [7/10], Loss: 13.960562705993652
Epoch [8/10], Loss: 14.434782028198242
Epoch [9/10], Loss: 10.428662300109863
Epoch [10/10], Loss: 17.28628921508789


In [41]:
for epoch in range(num_epochs):
    for batch in data_loader:
        print(type(batch))
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

<class 'list'>


TypeError: linear(): argument 'input' (position 1) must be Tensor, not list

# Evaluation

In [ ]:
#todo